# Обучим модели для классификации отзывов

In [73]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import joblib
import json 

from collections import Counter
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from functions.util_func import data_preprocessing, encode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Загружаем датасет и выбиарем колнки с отзывом и оценкой
df = pd.read_json('../kinopoisk.jsonl', lines=True)
print(df.columns)
df = df[['grade3', 'content']]
# df.head()

Index(['part', 'movie_name', 'review_id', 'author', 'date', 'title', 'grade3',
       'grade10', 'content'],
      dtype='object')


In [59]:
# Преобработка текста для bag of words and RNN models
df["cleaned_content"] = df["content"].apply(data_preprocessing)
# Encode label
df['label'] = df["grade3"].apply(encode)
df.head()

,grade3,content,label,cleaned_content
0,Good,"\n""Блеф» — одна из моих самых любимых комедий....",2,блеф» — одн мо сам любим комед фильм наверн см...
1,Good,\nАдриано Челентано продолжает радовать нас св...,2,адриа челента продолжа радова сво работ жизн к...
2,Good,"\nНесомненно, это один из великих фильмов 80-х...",2,несомнен эт велик фильм 80х год исключительн к...
3,Good,\nЭта фраза на мой взгляд отражает сюжет несом...,2,эт фраз взгляд отража сюжет несомнен прекрасн ...
4,Neutral,"\n- как пела Земфира, скорее всего, по соверше...",1,пел земфир скор совершен друг повод «фильм чел...


## TF-IDF and Bag of Words

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.8,# Игнорируем слова которые встречаются в 80% текстов
    min_df=10 # Игонорируем слова, которые встречаются меньше 10 раз
    )
vectorizer = vectorizer.fit(df.cleaned_content)
X = vectorizer.transform(df.cleaned_content)

# Сохраним vectorizer
joblib.dump(vectorizer, "vectorizer_tfidf.pkl")

print(X.shape)

(36591, 24712)


In [22]:
from sklearn.naive_bayes import MultinomialNB


In [123]:
# Строим модель без балансировки класов


# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.3)

# Обучение классификатора
classifier = MultinomialNB(alpha=0.01)
classifier.fit(X_train, y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print("f1:", f1_score(y_test, predictions, average='macro'))

Accuracy: 0.8058844962652578
f1: 0.5381581948150086


In [62]:
# Модель с балансировкой

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

# Получение весов классов
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Создание модели с взвешиванием классов
model = RandomForestClassifier(class_weight=dict(enumerate(class_weights)))
model.fit(X_train, y_train)

# Оценка классификатора
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print("f1:", f1_score(y_test, predictions, average='macro'))
# f1_sc = f1_score(y_test, predictions, average='macro')
Counter(predictions)


Accuracy: 0.7746401894698488
f1: 0.41384797020605335


Counter({2: 10572, 0: 381, 1: 25})

In [124]:
# Модель с уменьшением преобладающего класса

from imblearn.under_sampling import RandomUnderSampler

# Создание экземпляра RandomUnderSampler
rus = RandomUnderSampler()

# Применение уменьшения выборки к данным
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# Обучение классификатора
classifier = MultinomialNB()
classifier.fit(X_resampled, y_resampled)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print("f1:", f1_score(y_test, predictions, average='macro'))
print(Counter(predictions))

f1_sc = f1_score(y_test, predictions, average='macro')

# Save model
joblib.dump(classifier, 'MultinomialNB.pkl')

# Save f1-score
# scores = {}

with open('models_f1_scores.json', 'r') as f:
    scores = json.load(f)

with open('models_f1_scores.json', 'w') as f:
    scores['TF-IDF'] = f1_sc
    json.dump(scores,f)


Accuracy: 0.7724539989069047
f1: 0.6040766241019483
Counter({2: 7709, 0: 1975, 1: 1294})


In [125]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_resampled, y_resampled)

predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print("f1:", f1_score(y_test, predictions, average='macro'))
print(Counter(predictions))

Accuracy: 0.7724539989069047
f1: 0.6040766241019483
Counter({2: 7709, 0: 1975, 1: 1294})


In [66]:
# Проверим модель
from functions.util_func import predict

vect = joblib.load("vectorizer_tfidf.pkl")
clf = joblib.load("MultinomialNB.pkl")


text = df.query('label == 0')['content'].values[0]
print(text)
print('*'*10)
print(f'Клаccификация отзыва: ', predict(text, vectorizer=vect, classifier=clf))


Как же уныло… Я вынужден констатировать, что славная студия Дисней утратила моё доверие и попала в чёрный список студий, новые мультфильмы которых теперь я смотреть не буду. Признаться, в списке до этого были лишь ДримВоркс, но если учесть, что для меня в анимации существует только три крупных рыбы — Дисней, ДримВоркс и Пиксар, то надежда остаётся только на последних. Те, слава Богу, в последние годы делают исключительно шедевры или близко к ним, так что можно не беспокоиться. 

Ближе к «Рапунцель». Во время просмотра у меня постоянно копились новые претензии, которые я впоследствии забывал по причине их обильности. Поэтому я пойду проверенной схемой, поочередно обозвав плохими словами сюжет, юмор и персонажей. Начнём.

I. Сюжет.

Герои движутся из точки А в точку Б, попутно зарабатывая приключения на свою пятую точку. У меня сразу возникает дежавю, потому что тоже самое я писал в своём отзыве на «Ледниковый период». Ну мультфильмов с такой концепцией — миллионы. Другое дело, что я та

## RNN model

In [32]:
# Создадим токены
# Обычно при токенизации и сопоставлении целых чисел словам принцип такой: наиболее популярным словам соответсвуют меньшие индексы (но ноль зарезервирован – `pad`)

from collections import Counter

corpus = [word for text in df['cleaned_content'] for word in text.split()]
count_words = Counter(corpus)
# print(count_words)
sorted_words = count_words.most_common()


In [33]:
# Можем взять только слова, которые чаще всего встречаются в данных, 
# но это гиперпараметр
def get_words_by_freq(sorted_words: list, n: int = 10) -> list:
    return list(filter(lambda x: x[1] > n, sorted_words))

In [34]:
sorted_words = get_words_by_freq(sorted_words, 10)
sorted_words[-10:]

[('таганрог', 11),
 ('слепак', 11),
 ('скетчком', 11),
 ('денисовн', 11),
 ('«беременного»', 11),
 ('шплинт', 11),
 ('юнгвальдахилькевич', 11),
 ('ширвиндт', 11),
 ('gps', 11),
 ('навигатор', 11)]

In [35]:
# Каждому слову сопоставим целое число для дальнейшей передачи в нейросеть
vocab_to_int = {w: i+1 for i, (w,c) in enumerate(sorted_words)}
# save vocab
with open("vocab_for_GRU.json", "w") as outfile: 
    json.dump(vocab_to_int, outfile)

In [36]:
d= open("vocab_for_GRU.json")
json.load(d)

{'фильм': 1,
 '—': 2,
 'эт': 3,
 'котор': 4,
 'сво': 5,
 'сам': 6,
 'очен': 7,
 '10': 8,
 'прост': 9,
 'так': 10,
 'жизн': 11,
 'одн': 12,
 'актер': 13,
 'друг': 14,
 'главн': 15,
 'человек': 16,
 'люд': 17,
 'кажд': 18,
 'рол': 19,
 'картин': 20,
 'все': 21,
 'перв': 22,
 'геро': 23,
 'сюжет': 24,
 'истор': 25,
 'кин': 26,
 'игр': 27,
 'врем': 28,
 'просмотр': 29,
 'мир': 30,
 'сказа': 31,
 'говор': 32,
 'имен': 33,
 'зрител': 34,
 'лучш': 35,
 'режиссер': 36,
 'хорош': 37,
 'персонаж': 38,
 'хот': 39,
 'всем': 40,
 'част': 41,
 'получ': 42,
 'люб': 43,
 'сто': 44,
 'наш': 45,
 'момент': 46,
 'сдела': 47,
 'мног': 48,
 'лиш': 49,
 'слов': 50,
 'дел': 51,
 'интересн': 52,
 'работ': 53,
 'нов': 54,
 'больш': 55,
 'смотрет': 56,
 'нужн': 57,
 'сцен': 58,
 'мо': 59,
 'показа': 60,
 'конц': 61,
 'нам': 62,
 'дума': 63,
 'стал': 64,
 'мест': 65,
 'чувств': 66,
 'сыгра': 67,
 'год': 68,
 'хочет': 69,
 'смотр': 70,
 'понрав': 71,
 'вообщ': 72,
 'образ': 73,
 'настоя': 74,
 'мог': 75,
 'отличн

In [38]:
# Переводим отзывы в "числовой формат"
reviews_int = []
for text in df['cleaned_content']:
    r = [vocab_to_int[word] for word in text.split() if vocab_to_int.get(word)]
    reviews_int.append(r)
# reviews_int[:2]

In [39]:
# Посчитаем количество слов в отзыве
review_len = [len(x) for x in reviews_int]
df['Review len'] = review_len
df.head()

,grade3,content,cleaned_content,label,Review len
0,Good,"\n""Блеф» — одна из моих самых любимых комедий....",блеф» — одн мо сам любим комед фильм наверн см...,2,207
1,Good,\nАдриано Челентано продолжает радовать нас св...,адриа челента продолжа радова сво работ жизн к...,2,103
2,Good,"\nНесомненно, это один из великих фильмов 80-х...",несомнен эт велик фильм 80х год исключительн к...,2,35
3,Good,\nЭта фраза на мой взгляд отражает сюжет несом...,эт фраз взгляд отража сюжет несомнен прекрасн ...,2,109
4,Neutral,"\n- как пела Земфира, скорее всего, по соверше...",пел земфир скор совершен друг повод «фильм чел...,1,203


In [40]:
print(df['Review len'].describe())

count    36591.000000
mean       185.889126
std        124.685720
min          2.000000
25%         99.000000
50%        154.000000
75%        239.000000
max       1748.000000
Name: Review len, dtype: float64


In [41]:
from functions.util_func import padding

In [42]:
SEQ_LEN = 154
features = padding(reviews_int, SEQ_LEN)
print(features[1, :])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0  7000  4792   675  1064     5    53    11    26  4706
   555   907    54   407    52   451   345    25  2046   120   411  7149
  8735    30    44   740   330  5661    16     4  2201  5232  6390  1831
  7149  3052   256    85    11  5132     5   429  1138   209     1  2329
  1324  1315   233     4   279  2936  1712    27    13  1911  9006   887
   304    69  4769 11631    63    49   487    27   217    13   110  5448
  1114  2021    91   170   905    20   533   887   330  1020    46   140
   351   442   236  1463  4017   330   221    44    63   142    39  2614
    53   517     2   280 17993     3    57   564   167  7385]


In [43]:
# Создадим датасеты

X_train, X_valid, y_train, y_valid = train_test_split(features, df['label'].to_numpy(), test_size=0.2, random_state=1)

In [44]:
import torch
from torch.utils.data import TensorDataset, DataLoader


In [45]:
# create tensor dataset
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train).type(torch.LongTensor))
valid_data = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid).type(torch.LongTensor))

# dataloaders
BATCH_SIZE = 32
train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE)

In [46]:
# посмотрим, что внутри
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: BATCH_SIZE x SEQ_LEN', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample input: \n', sample_y)

Sample input size: BATCH_SIZE x SEQ_LEN torch.Size([32, 154])
Sample input: 
 tensor([[    0,     0,     0,  ...,   441,   334,     8],
        [  371,     1,    71,  ...,   589,   284,   113],
        [    1,  1652,     7,  ...,   155,   392,    12],
        ...,
        [  190,    11,   418,  ...,   202, 21790,  1455],
        [  556,   426,   143,  ...,     3,  1215,  1063],
        [  474,   809,   947,  ...,   869, 16777,   117]])
Sample input: 
 tensor([1, 2, 2, 1, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 0,
        2, 2, 0, 2, 0, 2, 2, 2])


In [47]:
from functions.util_func import RNN_config


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

config = RNN_config(
    vocab_size=len(vocab_to_int)+1,
    device=DEVICE,
    n_layers=1,
    embedding_dim=16,
    hidden_size=32,
    seq_len=SEQ_LEN,
    bidirectional=False
)

config

RNN_config(vocab_size=24927, device='cuda', n_layers=1, embedding_dim=16, hidden_size=32, seq_len=154, bidirectional=False)

In [91]:
import torch.nn as nn
import torchutils as tu

class GRUnet(nn.Module):
    def __init__(self, rnn_conf = config) -> None:
        super().__init__()
        self.rnn_conf = rnn_conf
        self.seq_len    = rnn_conf.seq_len 
        self.emb_size   = rnn_conf.embedding_dim 
        self.hidden_dim = rnn_conf.hidden_size
        self.n_layers   = rnn_conf.n_layers
        self.vocab_size = rnn_conf.vocab_size
        self.bidirectional = bool(rnn_conf.bidirectional)

        self.embedding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.emb_size)

        self.bidirect_factor = 2 if self.bidirectional == 1 else 1
        
        self.gru = nn.GRU(
            input_size=self.emb_size,
            hidden_size=self.hidden_dim,
            num_layers=self.n_layers,
            batch_first=True,
            bidirectional=self.bidirectional,
            # device=rnn_conf.device
            )
        
        self.linear = nn.Sequential(
            nn.Linear(self.hidden_dim*self.seq_len*self.bidirect_factor, 128),
            nn.Tanh(),
            nn.Linear(128,32),
            nn.Tanh(),
            nn.Linear(32,3)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x.to(self.rnn_conf.device)) # Пернесим данные на девайс и создаем ембеддинги
        output, _ = self.gru(x) # Забираем hidden states со всех промежуточных состояний, второй выход финального слоя отправляем в _
        output = output.reshape(output.shape[0], -1)
        out = self.linear(output.squeeze())
        return out

model_gru = GRUnet(config)
model_gru.to(DEVICE)
tu.get_model_summary(model_gru, sample_x.to(config.device))

    


Layer                 Kernel         Output       Params          FLOPs
0_embedding         [16, 24927]   [32, 154, 16]   398,832         4,928
1_gru                         -   [32, 154, 32]     4,800   200,273,920
2_linear.Linear_0   [4928, 128]       [32, 128]   630,912    40,366,080
3_linear.Tanh_1               -       [32, 128]         0        20,480
4_linear.Linear_2     [128, 32]        [32, 32]     4,128       261,120
5_linear.Tanh_3               -        [32, 32]         0         5,120
6_linear.Linear_4       [32, 3]         [32, 3]        99         6,048
Total params: 1,038,771
Trainable params: 1,038,771
Non-trainable params: 0
Total FLOPs: 240,937,696 / 240.94 MFLOPs
-----------------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 1.88
Params size (MB): 3.96
Estimated Total Size (MB): 5.88


In [92]:
# from torchmetrics.classification import MulticlassAccuracy
from torchmetrics.classification import F1Score

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_gru.parameters())
metric = F1Score(task="multiclass", num_classes=3, average='macro').to(DEVICE)

In [106]:
from functions.train_rnn import train

In [94]:
train_losses, val_losses, train_metric, val_metric, rnn_time = train(
    epochs=2, 
    model=model_gru, 
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer,
    rnn_conf=config,
    criterion=criterion,
    metric=metric
    )

Epoch 1
train_loss : 0.6316 val_loss : 0.5356
train_accuracy : 0.39 val_accuracy : 0.50
Epoch 2
train_loss : 0.4686 val_loss : 0.5131
train_accuracy : 0.56 val_accuracy : 0.56


In [95]:
# Save metrics
with open('models_f1_scores.json', 'r') as f:
    scores = json.load(f)

with open('models_f1_scores.json', 'w') as f:
    scores['GRU'] = val_metric[-1]
    json.dump(scores,f)

In [96]:
# Save model

torch.save(model_gru, 'model_gru.pt')

In [ ]:
# test_model

In [97]:
from functions.util_func import predict_gru


In [143]:
text = df.query('label == 0')['content'].values[0]
print(text)

predict_gru(text)


Как же уныло… Я вынужден констатировать, что славная студия Дисней утратила моё доверие и попала в чёрный список студий, новые мультфильмы которых теперь я смотреть не буду. Признаться, в списке до этого были лишь ДримВоркс, но если учесть, что для меня в анимации существует только три крупных рыбы — Дисней, ДримВоркс и Пиксар, то надежда остаётся только на последних. Те, слава Богу, в последние годы делают исключительно шедевры или близко к ним, так что можно не беспокоиться. 

Ближе к «Рапунцель». Во время просмотра у меня постоянно копились новые претензии, которые я впоследствии забывал по причине их обильности. Поэтому я пойду проверенной схемой, поочередно обозвав плохими словами сюжет, юмор и персонажей. Начнём.

I. Сюжет.

Герои движутся из точки А в точку Б, попутно зарабатывая приключения на свою пятую точку. У меня сразу возникает дежавю, потому что тоже самое я писал в своём отзыве на «Ледниковый период». Ну мультфильмов с такой концепцией — миллионы. Другое дело, что я та

('Bad', 0.04)

## BERT

In [21]:
# Загружаем датасет и выбиарем колнки с отзывом и оценкой
df = pd.read_json('../kinopoisk.jsonl', lines=True)
print(df.columns)
df = df[['grade3', 'content']]
# df.head()
# Encode label
df['label'] = df["grade3"].apply(encode)
df.head()

Index(['part', 'movie_name', 'review_id', 'author', 'date', 'title', 'grade3',
       'grade10', 'content'],
      dtype='object')


,grade3,content,label
0,Good,"\n""Блеф» — одна из моих самых любимых комедий....",2
1,Good,\nАдриано Челентано продолжает радовать нас св...,2
2,Good,"\nНесомненно, это один из великих фильмов 80-х...",2
3,Good,\nЭта фраза на мой взгляд отражает сюжет несом...,2
4,Neutral,"\n- как пела Земфира, скорее всего, по соверше...",1


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
import torch
from torch import nn
import matplotlib.pyplot as plt

# импортируем трансформеры
import transformers
import warnings
warnings.filterwarnings('ignore')

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [23]:
# DistilBERT:

## задаем саму модель
model_class = transformers.DistilBertModel

# токенайзер к ней (для некоторых моделей токенайзер будет отличаться, см.
## в документации к каждой модели конкретно)
tokenizer_class = transformers.DistilBertTokenizer

## WordPiece
## BPE - Byte pair encoding

## загружаем веса для моделей
pretrained_weights = 'distilbert-base-uncased' # tiny -> base-> large -> XL

###########################################
# создаем объекты токенизатора для и модели
tokenizer = tokenizer_class.from_pretrained('distilbert-base-uncased')
model = model_class.from_pretrained('distilbert-base-uncased')

In [24]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [25]:
## зададим максимальную длину последовательности
MAX_LEN = 154

In [26]:
# токенизируем одну фразу
tokenizer.encode(
    'Hi book film monkey',
    add_special_tokens=True,
    truncation=True,
    max_length=MAX_LEN,
    # padding='max_length'
    )

[101, 7632, 2338, 2143, 10608, 102]

In [56]:
# применяем токенизатор:
# -≥ add_special_tokens = добавляем служебные токены (CLS=101, SEP=102)
# -≥ truncation = обрезаем по максимальной длине
# -≥ max_length = максимальная длина последовательности
tokenized = df['content'].apply((lambda x: tokenizer.encode(x,
                                                                      add_special_tokens=True,
                                                                      truncation=True,
                                                                      max_length=MAX_LEN)))
tokenized

0        [101, 1000, 1181, 29436, 15290, 29749, 1090, 1...
1        [101, 1180, 29742, 16856, 10325, 28995, 14150,...
2        [101, 1192, 15290, 29747, 14150, 29745, 18947,...
3        [101, 1208, 22919, 10260, 1199, 16856, 10260, ...
4        [101, 1011, 1189, 10260, 23925, 1194, 15290, 2...
                               ...                        
36586    [101, 1192, 29748, 1196, 1202, 15290, 29741, 1...
36587    [101, 1191, 14150, 29743, 18947, 14150, 1192, ...
36588    [101, 1199, 10325, 29436, 23742, 29745, 1194, ...
36589    [101, 2385, 1196, 15290, 18947, 22919, 17432, ...
36590    [101, 1208, 29750, 1010, 1189, 10260, 23925, 1...
Name: content, Length: 36591, dtype: object

In [57]:
# чтобы дополнить последовательности до фиксированной длины, допишем нули к
# коротким последовательностям
padded = np.array([i + [0]*(MAX_LEN-len(i)) for i in tokenized.values])

In [58]:
# чтобы attention не считался для позиций, где установлены фиктивные
# нулевые токены, мы делаем для него маску
attention_mask = np.where(padded != 0, 1, 0)
attention_mask[1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [59]:
print(padded.shape); print(attention_mask.shape)

(36591, 154)
(36591, 154)


In [60]:
class BertInputs(torch.utils.data.Dataset):
    def __init__(self, tokenized_inputs, attention_masks):
        super().__init__()
        self.tokenized_inputs = tokenized_inputs
        self.attention_masks = attention_masks

    def __len__(self):
        return self.tokenized_inputs.shape[0]

    def __getitem__(self, idx):
        ids = self.tokenized_inputs[idx]
        ams = self.attention_masks[idx]

        return ids, ams

dataset = BertInputs(padded, attention_mask)

In [61]:
loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=False)
sample_ids, sample_ams = next(iter(loader))
print(sample_ids.shape, sample_ams.shape)

torch.Size([10, 154]) torch.Size([10, 154])


In [33]:
sample_ids

tensor([[  101,  1000,  1181,  ..., 29754, 10325,   102],
        [  101,  1180, 29742,  ..., 10325, 15290,   102],
        [  101,  1192, 15290,  ..., 22919, 16856,   102],
        ...,
        [  101,  1199, 10325,  ..., 29113, 29744,   102],
        [  101,  1182, 15290,  ..., 10325, 29747,   102],
        [  101,  1192, 10260,  ...,  1194, 16856,   102]])

In [64]:
model.cuda();

In [65]:
%%time
features = []
for inputs, attention_masks in loader:
# Получаем выход модели (нам оттуда нужно не все)
    with torch.inference_mode():
        last_hidden_states = model(inputs.cuda(), attention_mask=attention_masks.cuda())
        vectors = last_hidden_states[0][:,0,:].cpu().numpy()
    features.extend(vectors)
len(features)

CPU times: user 4min 38s, sys: 1.93 s, total: 4min 40s
Wall time: 4min 40s


36591

In [66]:
X_train, X_val, y_train, y_val = train_test_split(features, df['label'], stratify=df['label'])
print(f'Features shape: {len(X_train)}, Target shape: {len(y_train)}')

Features shape: 27443, Target shape: 27443


In [67]:
df['label'].value_counts()/len(features)*100

label
2    74.510125
0    12.984067
1    12.505807
Name: count, dtype: float64

In [68]:
from imblearn.under_sampling import RandomUnderSampler

# Создание экземпляра RandomUnderSampler
rus = RandomUnderSampler()

# Применение уменьшения выборки к данным
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

In [71]:
Counter(y_resampled)

Counter({0: 3432, 1: 3432, 2: 3432})

In [74]:
# Обучение модели 
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_resampled, y_resampled)

# Оценка классификатора
predictions = clf.predict(X_val)
print("Accuracy:", accuracy_score(y_val, predictions))
print("f1:", f1_score(y_val, predictions, average='macro'))
print(Counter(predictions))
f1_sc = f1_score(y_val, predictions, average='macro')


# Save model
joblib.dump(clf, 'LogReg_Bert.pkl')

# Save f1-score
with open('models_f1_scores.json', 'r') as f:
    scores = json.load(f)

with open('models_f1_scores.json', 'w') as f:
    scores['LogReg_Bert'] = f1_sc
    json.dump(scores,f)

Accuracy: 0.4490599038041102
f1: 0.35641602474323064
Counter({2: 3936, 0: 2956, 1: 2256})


In [3]:
# test predict

## задаем саму модель
model_class = transformers.DistilBertModel
# токенайзер к ней (для некоторых моделей токенайзер будет отличаться, см.
## в документации к каждой модели конкретно)
tokenizer_class = transformers.DistilBertTokenizer
## загружаем веса для моделей
pretrained_weights = 'distilbert-base-uncased' # tiny -> base-> large -> XL
# создаем объекты токенизатора для и модели
tokenizer = tokenizer_class.from_pretrained('distilbert-base-uncased')
model = model_class.from_pretrained('distilbert-base-uncased')



In [76]:

# применяем токенизатор:
text = 'фильм просто противно смотреть'
tokenized = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=MAX_LEN)
tokenized

# чтобы дополнить последовательности до фиксированной длины, допишем нули к
# коротким последовательностям
padded = np.array(tokenized + [0]*(MAX_LEN-len(tokenized)))

# чтобы attention не считался для позиций, где установлены фиктивные
# нулевые токены, мы делаем для него маску
attention_mask = np.where(padded != 0, 1, 0)
attention_mask


# # Функция предсказания BERT модели
# def prefict_bert(text:str, model)-> str:

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

('Neutral', 0.13)

In [77]:
attention_mask = torch.LongTensor(attention_mask).unsqueeze(0)
padded = torch.LongTensor(padded).unsqueeze(0)
padded

tensor([[  101,  1199, 10325, 29436, 23742, 29745,  1194, 16856, 14150, 29747,
         22919, 14150,  1194, 16856, 14150, 22919, 10325, 25529, 18947, 14150,
          1196, 29745, 14150, 22919, 16856, 15290, 22919, 23742,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [78]:
model.cpu()
features = []
with torch.inference_mode():
        last_hidden_states = model(padded, attention_mask=attention_mask)
        vectors = last_hidden_states[0][:,0,:].numpy()
        features = vectors

In [79]:
features.squeeze()

array([-2.44874671e-01,  8.14346671e-02, -1.12671301e-01, -3.86535257e-01,
       -5.81587590e-02, -2.34020427e-02,  2.35774994e-01,  2.28222519e-01,
       -2.53098249e-01, -2.65456676e-01, -4.38449271e-02, -8.84315893e-02,
       -2.86098212e-01,  9.54462737e-02, -1.83469206e-01,  1.70412958e-01,
       -6.58771768e-02,  4.89517413e-02,  1.93631589e-01, -8.58901739e-02,
       -1.40272990e-01, -1.66548774e-01,  1.23467213e-02, -1.00629516e-02,
       -4.32168782e-01, -4.55683246e-02, -1.98923528e-01,  2.45535910e-01,
        5.93585931e-02,  1.74247876e-01, -5.27213700e-02,  1.95643976e-01,
       -9.63049456e-02,  1.83278620e-01,  1.33451283e-01,  2.17306092e-01,
        1.07032582e-01, -1.11143313e-01,  1.52468815e-01, -1.51353434e-01,
        1.30054178e-02,  2.32798025e-01, -2.25140303e-01, -3.47766548e-01,
        2.22917303e-01, -3.11797053e-01, -2.47316766e+00,  3.62427860e-01,
       -3.53537947e-01, -4.43214148e-01,  3.35174590e-01,  5.95794898e-03,
        1.73638463e-01, -

In [75]:
clf = joblib.load("LogReg_Bert.pkl")
clf

LogisticRegression(random_state=0)

In [81]:
out = clf.predict(features)
from functions.util_func import decode
decode(out)


'Neutral'

In [84]:
from functions.util_func import predict_bert
predict_bert(text = text, BERT_model = model, tokenizer=tokenizer,clf_model=clf)

('Neutral', 0.07)